In [1]:
!nvidia-smi

Thu May 12 23:14:41 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 470.57.02    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Quadro RTX 8000     On   | 00000000:2F:00.0 Off |                    0 |
| N/A   53C    P0    61W / 250W |   3920MiB / 45556MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
path_to_json_file = 'Reddit_Twitter_dataset/twitter/sarcasm_detection_shared_task_twitter_training.jsonl'

In [3]:
import json 
di = dict()

with open(path_to_json_file, 'r') as j:
     json_data = [json.loads(line) for line in j]

i=0
for tweet in json_data:
    tot_cont=""
    for con in tweet['context']:
      tot_cont+=(con + " ")
    tot_cont+=tweet['response']
    lab = 0
    if tweet['label'] == 'SARCASM':
        lab=1
    d1 = {'text': tot_cont, 'label': lab}

    di[i] = d1
    i+=1

In [4]:
print(type(di))
print(di[0])

<class 'dict'>
{'text': "A minor child deserves privacy and should be kept out of politics . Pamela Karlan , you should be ashamed of your very angry and obviously biased public pandering , and using a child to do it . @USER If your child isn't named Barron ... #BeBest Melania couldn't care less . Fact . 💯 @USER @USER @USER I don't get this .. obviously you do care or you would've moved right along .. instead you decided to care and troll her ..", 'label': 1}


In [5]:
from transformers import CanineTokenizer, CanineForSequenceClassification

tokenizer = CanineTokenizer.from_pretrained("google/canine-s")

Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.


In [6]:
tokenized_datasets = []
for i in di:
  l = tokenizer(di[i]["text"], padding="max_length", truncation=True)
  l['labels'] = di[i]["label"]
  tokenized_datasets.append(l)

In [7]:
import random
import numpy as np
import torch 
def set_seed(seed):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

set_seed(42)

In [8]:
random.shuffle(tokenized_datasets)

train_data = tokenized_datasets[:4500]
val_data = tokenized_datasets[4500:]

In [9]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [10]:
model = CanineForSequenceClassification.from_pretrained("google/canine-s", num_labels=2)

Some weights of CanineForSequenceClassification were not initialized from the model checkpoint at google/canine-s and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)


    return {"accuracy": accuracy}

In [12]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="test_trainer/new_ep/copy_twitter",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=15,
    weight_decay=0.01,
    evaluation_strategy='epoch',
    save_strategy='epoch'
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=val_data,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

trainer.train()

/home/mss9240/.local/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 4500
  Num Epochs = 15
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 4230


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.502741,0.762000
2,0.535100,0.468240,0.778000
3,0.535100,0.557148,0.742000
4,0.321800,0.684472,0.770000
5,0.321800,0.849788,0.790000
6,0.151800,1.042805,0.792000
7,0.151800,1.135417,0.766000
8,0.090900,1.324319,0.792000
9,0.037200,1.268973,0.790000
10,0.037200,1.365285,0.786000


***** Running Evaluation *****
  Num examples = 500
  Batch size = 16
Saving model checkpoint to test_trainer/new_ep/copy_twitter/checkpoint-282
Configuration saved in test_trainer/new_ep/copy_twitter/checkpoint-282/config.json
Model weights saved in test_trainer/new_ep/copy_twitter/checkpoint-282/pytorch_model.bin
tokenizer config file saved in test_trainer/new_ep/copy_twitter/checkpoint-282/tokenizer_config.json
Special tokens file saved in test_trainer/new_ep/copy_twitter/checkpoint-282/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 500
  Batch size = 16
Saving model checkpoint to test_trainer/new_ep/copy_twitter/checkpoint-564
Configuration saved in test_trainer/new_ep/copy_twitter/checkpoint-564/config.json
Model weights saved in test_trainer/new_ep/copy_twitter/checkpoint-564/pytorch_model.bin
tokenizer config file saved in test_trainer/new_ep/copy_twitter/checkpoint-564/tokenizer_config.json
Special tokens file saved in test_trainer/new_ep/copy_twitter/c

TrainOutput(global_step=4230, training_loss=0.14062255808365823, metrics={'train_runtime': 4390.831, 'train_samples_per_second': 15.373, 'train_steps_per_second': 0.963, 'total_flos': 8.86724195328e+16, 'train_loss': 0.14062255808365823, 'epoch': 15.0})

In [72]:
test_file = 'Reddit_Twitter_dataset/twitter/sarcasm_detection_shared_task_twitter_testing.jsonl'

di_test = dict()

with open(test_file, 'r') as j:
     test_data = [json.loads(line) for line in j]

i=0
for tweet in test_data:
    tot_cont=""
    for con in tweet['context']:
      tot_cont+=(con + " ")
    tot_cont+=tweet['response']
    lab = 0
    if tweet['label'] == 'SARCASM':
        lab=1
    d1 = {'text': tot_cont, 'label': lab}

    di_test[i] = d1
    i+=1

    
tokenized_datasets_test = []
for i in di_test:
  l = tokenizer(di_test[i]["text"], padding="max_length", truncation=True)
  l['labels'] = di_test[i]["label"]
  tokenized_datasets_test.append(l)

In [73]:
model_path = "test_trainer/new_ep/copy_twitter/4ep_best_run"
model = CanineForSequenceClassification.from_pretrained(model_path, num_labels=2)

loading configuration file test_trainer/new_ep/copy_twitter/4ep_best_run/config.json
Model config CanineConfig {
  "_name_or_path": "google/canine-s",
  "architectures": [
    "CanineForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 57344,
  "downsampling_rate": 4,
  "eos_token_id": 57345,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "local_transformer_stride": 128,
  "max_position_embeddings": 16384,
  "model_type": "canine",
  "num_attention_heads": 12,
  "num_hash_buckets": 16384,
  "num_hash_functions": 8,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.16.2",
  "type_vocab_size": 16,
  "upsampling_kernel_size": 4,
  "use_cache": true
}

loading weights file test_trainer/new_ep/copy_twitter/4ep_best_run/pytorch_model.bin
Al

In [77]:
test_trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=val_data,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)


PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [78]:
test_trainer.evaluate(eval_dataset = tokenized_datasets_test)

{'eval_loss': 0.8127884864807129,
 'eval_accuracy': 0.7287853577371048,
 'eval_runtime': 15.9934,
 'eval_samples_per_second': 37.578,
 'eval_steps_per_second': 2.376}